In [ ]:
import tensorflow as tf

In [ ]:
#for densenet-bc uncomment the lines of this block
def dense_block(inps, filter, times, id):
  for time in range(0, times):
    #inps = BatchNormalization()(inps)
    shape = inps.shape
    part1 = inps
    part2 = part1
    part2 = tf.keras.layers.Conv2D(filter[0], 1, padding='same', activation='relu')(part2)
    #part2 = BatchNormalization()(part2)
    part2 = tf.keras.layers.Conv2D(filter[1], 3, padding='same', activation='relu')(part2)
    inps = tf.keras.layers.Concatenate()([part1, part2])
  return inps

In [ ]:
def conv_block(inps, convs):
  x = inps
  for conv in convs:
	  x = tf.keras.layers.Conv2D(conv['filter'], conv['kernel'], conv['strides'], conv['padding'], activation='relu')(x)
  return x

In [ ]:
def nin_block(inps, filter, ker):
  x = inps
  x = tf.keras.layers.Conv2D(filter, ker,padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(filter, 1,padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(filter, 1,padding='same', activation='relu')(x)
  return x

In [ ]:
#for densenet block remains same
#for densenet-b also the block remains same
#for densenet-bc, while calling all conv_block,
#change filters to half of used in dense_block
#In this case change 48/2 i.e 24
#np chnage to conv_block with layer_id = 0 
def densenet():
  input_image = tf.keras.layers.Input(shape = (224,224,3))
  #Layer 0 => 1
  x = conv_block(input_image, [{'filter':32, 'kernel':7, 'padding':'same', 'strides' : 2, 'layer_ids':0}])
  x = tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2))(x)

  #Dense block 1
  x = dense_block(x, [32, 32], 10, 'dense_block1_')
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = conv_block(x, [{'filter':32, 'kernel':1,'strides' : 1, 'padding':'same', 'layer_ids':2}])
  x = tf.keras.layers.AveragePooling2D(pool_size = (2,2), strides = (2,2))(x)
  
  #dense block 2
  x = dense_block(x, [32, 32], 10, 'dense_block2_')
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = conv_block(x, [{'filter':32, 'kernel':1,'strides': 1, 'padding':'same', 'layer_ids':3}])
  x = tf.keras.layers.AveragePooling2D(pool_size = (2,2), strides = 2)(x)
  
  #dense block 3
  x = dense_block(x, [32, 32], 10, 'dense_block3_')
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = conv_block(x, [{'filter':32, 'kernel':1, 'strides' : 1, 'padding':'same', 'layer_ids':4}])
  x = tf.keras.layers.AveragePooling2D(pool_size = (2,2), strides = 2)(x)
  
  #nin block
  x = nin_block(x, 10, 3)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Reshape((1, 1, 10))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.models.Model(inputs = input_image, outputs = x)
  return x
 

In [ ]:
model_densenet = densenet()
model_densenet.summary()

In [ ]:
tf.keras.utils.plot_model(model_densenet, to_file = 'model_densenet.png', show_shapes=True, show_layer_names=True)